## RNN for char/word level language modeling
refer to:https://github.com/uyaseen/theano-recurrence

### prepare data

In [3]:
import nltk
import itertools
import getpass
import numpy as np

In [6]:
def read_char_data(f_name, seq_length):
    data = open(f_name, 'r').read()
    print (type(data))
    voc = list(set(data))
    print (voc)
    print('data size: %i, vocab size: %i' % (len(data), len(voc)))
    chars_to_ix = {ch: i for i, ch in enumerate(voc)}
    ix_to_chars = {i: ch for i, ch in enumerate(voc)}
    x_ = [chars_to_ix[ch] for ch in data]
    y_ = x_[1:] + x_[:1]
    x = []
    y = []
    e_idx = seq_length
    for i in xrange(0, len(x_) - 1, seq_length):
        if len(x_[i:e_idx]) == seq_length:
            x.append(x_[i:e_idx])
            y.append(y_[i:e_idx])
            e_idx += seq_length
    return [(x, y), (voc, ix_to_chars, chars_to_ix)]

def read_word_data(f_name, seq_length):
    data = tokenize(open(f_name, 'r').read())
    voc = list(set(data))
    print('data size: %i, vocab size: %i' % (len(data), len(voc)))
    words_to_ix = {wd: i for i, wd in enumerate(voc)}
    ix_to_words = {i: wd for i, wd in enumerate(voc)}
    x_ = [words_to_ix[wd] for wd in data]
    y_ = x_[1:] + x_[:1]
    x = []
    y = []
    e_idx = seq_length
    for i in xrange(0, len(x_) - 1, seq_length):
        if len(x_[i:e_idx]) == seq_length:
            x.append(x_[i:e_idx])
            y.append(y_[i:e_idx])
            e_idx += seq_length
    return [(x, y), (voc, ix_to_words, words_to_ix)]

def tokenize(text):
    ll = [[nltk.word_tokenize(w), ' '] for w in text.split()]
    return list(itertools.chain(*list(itertools.chain(*ll))))

In [7]:
f_name = '/home/'+getpass.getuser()+'/git_test/test_data/examples/machine_translation/input.txt'
seq_length = 100
(trn_x, trn_y), (vocabulary, idx_to_words, words_to_idx) = read_word_data(f_name, seq_length)
trn_x = np.asarray(trn_x, dtype=np.float32)
trn_y = np.asarray(trn_y, dtype=np.float32)
print (trn_x.shape, trn_y.shape)

data size: 2010, vocab size: 481
((20, 100), (20, 100))


### build model

In [8]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation, Embedding

In [9]:
vocab_size = len(vocabulary)
emb_dim = 100
hidden_dim = 100
output_dim = len(vocabulary)
model = Sequential()
model.add(Embedding(output_dim=emb_dim, input_dim=vocab_size))

In [15]:
print model.layers[0].trainable_weights
print model.layers[0].non_trainable_weights


abc
[]


In [17]:
from keras.layers import Input, Dense
from keras.models import Model

In [19]:
a = Input(shape=(32, ))
b = Dense(16, activation='softmax')(a)
model = Model(input=a, output=b)
print (help(a))

Help on Tensor in module tensorflow.python.framework.ops object:

class Tensor(_TensorLike)
 |  Represents one of the outputs of an `Operation`.
 |  
 |  A `Tensor` is a symbolic handle to one of the outputs of an
 |  `Operation`. It does not hold the values of that operation's output,
 |  but instead provides a means of computing those values in a
 |  TensorFlow [`Session`](../../api_docs/python/client.md#Session).
 |  
 |  This class has two primary purposes:
 |  
 |  1. A `Tensor` can be passed as an input to another `Operation`.
 |     This builds a dataflow connection between operations, which
 |     enables TensorFlow to execute an entire `Graph` that represents a
 |     large, multi-step computation.
 |  
 |  2. After the graph has been launched in a session, the value of the
 |     `Tensor` can be computed by passing it to
 |     [`Session.run()`](../../api_docs/python/client.md#Session.run).
 |     `t.eval()` is a shortcut for calling
 |     `tf.get_default_session().run(t)`.
